## Holistic AI x UCL AI Society Hackathon Tutorial

### Track 1: Multi-Objective Optimization for AI Trustworthiness in Tabular Data Classification

<h1 align="center">
<img src="https://bmcnews.com.br/wp-content/uploads/2023/11/NOVO-PROJETO-27-1024x536.jpg" width="500">
<br>Study Case: The Minimum Wage Dataset
<h4 align="center">Technical Risks in Machine Learning Models</h2>
</h1>


The **Minimum Wage** dataset, specifically curated for the first edition of the HAI-UCL Hackathon, originates from Brazil's **RAIS** (Relação Anual de Informações Sociais) database. Compiled annually by the Brazilian Ministry of Labor, RAIS provides detailed records on formal employment, encompassing essential information such as worker demographics, wage levels, job tenure, and firm-specific details like location, industry classification, and company size. RAIS data enables rigorous labor economic analyses, making it crucial for studying income distribution, employment patterns, and social inequalities in Brazil.

The **Minimum Wage** dataset includes a carefully balanced sample of 100,000 workers, divided equally across four demographic groups: White-Male, White-Female, Black-Male, and Black-Female (each comprising 25%). This balanced design allows for focused analysis of wage disparities and demographic influences by race and gender, supporting insights into socioeconomic trends in the Brazilian labor market.

**(Goal)** The primary goal of analyzing the Minimum Wage dataset is to predict whether a worker’s average income falls above or below five times the minimum wage threshold.

**(Features)** The following table presents the 13 features of Minimum Wage dataset, including the target variable "y (target)".

| Feature                  | Description                                                                                      |
|--------------------------|--------------------------------------------------------------------------------------------------|
| `age` | Age of the individual in years. |
| `state` | Federal state of residence or work, typically coded as a two-letter abbreviation (e.g., RJ for Rio de Janeiro, SP for São Paulo). |
| `time_employed` | Duration of employment with the current employer, often measured in months or years. |
| `contracted_hours` | Contracted weekly working hours, indicating the standard hours agreed upon for the role. |
| `intermittent_work` | Indicator of intermittent work status (binary: 0 = "No", 1 = "Yes"), denoting sporadic or flexible employment arrangements. |
| `partial_time_work` | Indicator of part-time employment (binary: 0 = "No", 1 = "Yes"), typically under 40 hours per week. |
| `economic_sector` | Categorical field for the sector of the economy in which the individual is employed (e.g., agriculture, manufacturing). |
| `cbo_2002` | Occupational classification code based on the Brazilian Classification of Occupations (CBO 2002). |
| `age_group` | Categorical age group classification (e.g., 18-24, 25-34), often used for demographic analysis. |
| `education_level` | Highest level of education completed (e.g., primary, secondary, higher education). |
| `sex` | Sex of the individual (binary: "Male", "Female"). |
| `race` | Race/ethnicity classification, typically based on self-identification categories (e.g., White, Black, Pardo). |
| `disability` | Indicator of disability status (binary: 0 = "No", 1 = "Yes"), identifying individuals with disabilities. |
| `y (target)` | Target variable indicating whether the worker receives more than five minimum wages on average (binary: 0 = "No", 1 = "Yes"). |

## Setup and Dependencies
Install the necessary libraries to get started:

In [1]:
pip install holisticai scikit-learn pandas numpy matplotlib seaborn jax


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Load Dataset

In [3]:
from holisticai.datasets import load_dataset

dataset = load_dataset("mw_small", protected_attribute="race")
split_dataset = dataset.train_test_split(test_size=0.2, random_state=42)
train = split_dataset['train']
test = split_dataset['test']

NotImplementedError: 

## 1. Train a Binary Classification Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from holisticai.utils import BinaryClassificationProxy

model = RandomForestClassifier(random_state=42)

model.fit(train['X'], train['y'])

proxy = BinaryClassificationProxy(predict=model.predict, predict_proba=model.predict_proba)

## 2. Measuring Efficacy

Efficacy metrics are used to evaluate the performance of a model by quantifying how accurately it makes predictions.

## a) Accuracy

**Accuracy** is a metric that measures the proportion of correct predictions made by a classification model out of the total predictions. It is a commonly used metric for model evaluation, especially when the classes are balanced.

### Formula

Accuracy is defined as:

$$
\text{Accuracy} = \frac{\text{True Positives} + \text{True Negatives}}{\text{Total Predictions}}
$$

where:
- **True Positives (TP)**: Correct positive predictions.
- **True Negatives (TN)**: Correct negative predictions.
- **Total Predictions** $= \text{True Positives} + \text{True Negatives} + \text{False Positives} + \text{False Negatives}$.

### Interpretation

- **Accuracy = 1**: The model predicts all outcomes correctly.
- **Accuracy < 1**: The model has some incorrect predictions.

Accuracy is useful when the class distribution is balanced. However, it can be misleading if the data is imbalanced, as it does no


In [ ]:
from sklearn.metrics import accuracy_score

y_test_pred = proxy.predict(test['X'])
accuracy_score(test['y'], y_test_pred)

## b) F1 Score

The **F1 Score** is a metric that combines both precision and recall into a single score. It is especially useful for evaluating the performance of a classification model when there is an imbalance in the class distribution, as it balances the trade-off between false positives and false negatives.

### Formula

The F1 Score is the harmonic mean of precision and recall and is defined as:

$$
F1 = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}
$$

where:
- **Precision**: The ratio of correctly predicted positive observations to total predicted positives.

  $$
  \text{Precision} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}
  $$

- **Recall**: The ratio of correctly predicted positive observations to all actual positives.

  $$
  \text{Recall} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}
  $$

### Interpretation

- **F1 Score = 1**: Perfect precision and recall.
- **F1 Score = 0**: Model fails to identify any true positives.
  
A higher F1 Score indicates better

In [ ]:
from sklearn.metrics import f1_score

f1_score(test['y'], y_test_pred)

What other efficacy metrics do we have for binary classification? Holistic AI library allows you to perform a practical inspection of the metrics by calling ```classification_efficacy_metrics```:

In [ ]:
from holisticai.efficacy.metrics import classification_efficacy_metrics

classification_efficacy_metrics(test['y'], y_test_pred)

# 2. Measuring Bias

In machine learning fairness, Equal Opportunity and Equal Outcomes are distinct concepts:

- **Equal Opportunity**: Ensures individuals with similar qualifications have equal chances of positive outcomes, focusing on equal true positive rates across groups.

- **Equal Outcomes**: Aims for equal proportions of positive results among different demographic groups, regardless of individual qualifications.

## a) Disparate Impact

**Disparate Impact** is a fairness metric used to measure the relative rate of favorable outcomes between an unprivileged group and a privileged group. It is commonly used to assess whether a model’s predictions disproportionately favor one group over another. A value close to 1 indicates fairness, while values significantly above or below 1 indicate potential bias.

### Formula

$$
\text{Disparate Impact} = \frac{P(\text{positive outcome | unprivileged})}{P(\text{positive outcome | privileged})}
$$

where:
- $P(\text{positive outcome | unprivileged})$ is the probability of a positive outcome for the unprivileged group.
- $P(\text{positive outcome | privileged})$ is the probability of a positive outcome for the privileged group.


### Interpretation

- **Disparate Impact ≈ 1**: The model is fair (similar outcomes for both groups).
- **< 1**: The privileged group has a higher rate of positive outcomes.
- **> 1**: The unprivileged group has a higher rate of positive outcomes.

In practice, a common threshold for fairness is that **Disparate Impact should be between 0.8 and 1.25**. If the metric falls outside this range, the model may be considered biased.

In [ ]:
from holisticai.bias.metrics import disparate_impact

disparate_impact(test['group_a'], test['group_b'], y_test_pred)

## b) Equal Opportunity Difference

The **Equal Opportunity Difference** is a fairness metric that measures the difference in true positive rates (TPR) between privileged and unprivileged groups in a classification model. Ideally, this difference should be close to zero to ensure fairness.

### Formula

$$
\text{Equal Opportunity Difference} = TPR_{privileged} - TPR_{unprivileged}
$$

### Interpretation

- **0**: The model is fair (Equal Opportunity).
- **Positive**: The privileged group has a higher chance of a positive prediction.
- **Negative**: The unprivileged group has a higher chance of a positive prediction.

In [ ]:
from holisticai.bias.metrics import equal_opportunity_diff

equal_opportunity_diff(test['group_a'], test['group_b'], y_test_pred, test['y'])

You can access all bias metrics for binary classification available in the HolisticAI library using the ```classification_bias_metrics``` function.

In [ ]:
from holisticai.bias.metrics import classification_bias_metrics

classification_bias_metrics(test['group_a'], test['group_b'], y_test_pred, test['y'])

# Measuring Explainability

Model interpretability can be quantified through various criteria that assess how understandable a model is, as well as through transparent surrogate models that aim to represent the actual model and its decision-making process. Some criteria include:

- **Concentration of Feature Importance**: Evaluating how feature importance is distributed across the model.

- **Consistency of Feature Importance Ranking**: Assessing the stability of feature importance rankings across different decision groups within the model, such as classes 0 and 1 in binary classification.

- **Stability of Feature Importance Across the Dataset**: Analyzing how feature importance varies throughout the dataset.

- **Complexity of Feature Relationships**: Examining the complexity of the relationship between features and their partial dependence plots.

Another approach to quantifying interpretability is to evaluate how effectively a transparent model fits the actual model. For instance, if the surrogate model is a decision tree, additional measures such as tree depth or the number of features used by the transparent model can be considered.

In this tutorial, we will calculate certain model features that will assist in evaluating interpretability metrics.

In [ ]:
from holisticai.inspection import compute_permutation_importance
from holisticai.utils.surrogate_models import create_surrogate_model
from holisticai.inspection import compute_partial_dependence
from holisticai.inspection import compute_conditional_permutation_importance


# Feature Importance
importance = compute_permutation_importance(proxy, X=train['X'], y=train['y'])
top_importance = importance.top_alpha()
top_feature_names = top_importance.feature_names

# Conditional Feature Importance
conditional_impotance = compute_conditional_permutation_importance(proxy, X=test['X'], y=test['y'])

# Partial Dependence
X_sample = test['X'].sample(1000)
partial_dependence = compute_partial_dependence(features=top_feature_names, proxy=proxy, X=X_sample)

# Surrogate Model
surrogate_model = create_surrogate_model(X=train['X'], y_pred=train['y'], surrogate_type='tree', learning_task='classification')

Measuring Alpha  Score that represents the percentage of features that represent the 80% of importnaces in the model behaivours. A lower value points that we can explain the model decision with few features.

In [ ]:
from holisticai.explainability.metrics.global_feature_importance import alpha_score

alpha_score(importance)

Measuring Fluctuation Ratio points the percentage in the PDP curve that present fluctuation in the model prediction.

In [ ]:
from holisticai.explainability.metrics.global_feature_importance import fluctuation_ratio

fluctuation_ratio(partial_dependence)

Rank Alignment estimates the percentage of features that remain aligned at the top of importances for label 0 and label 1

In [ ]:
from holisticai.explainability.metrics.global_feature_importance import rank_alignment

rank_alignment(conditional_impotance, top_importance)

Global Feature Importance Based Metrics

In [ ]:
from holisticai.explainability.metrics import classification_global_feature_importance_explainability_metrics

classification_global_feature_importance_explainability_metrics(partial_dependence, importance, conditional_impotance)

Tree based Metrics

In [ ]:
from holisticai.explainability.metrics import tree_explainability_metrics

tree_explainability_metrics(surrogate_model)

Surrogate Based Metrics

In [ ]:
from holisticai.explainability.metrics import classification_surrogate_explainability_metrics

y_pred = proxy.predict(test['X'])
classification_surrogate_explainability_metrics(test['X'], test['y'], y_pred, surrogate_model)

# Measuring Security

Security metrics in machine learning evaluate the robustness and resilience of models against threats to data privacy, model integrity, and information leakage. They provide insight into how susceptible a model is to various risks and guide strategies to safeguard sensitive information and maintain reliable performance.

Here’s a brief description of each type of security metric in the library:

- **SHAPR**: This metric assesses membership privacy risk by estimating how much individual training data points influence model predictions. By using Shapley values, SHAPR quantifies the likelihood of data memorization, which can reveal vulnerabilities to membership inference attacks.

- **Data Minimizer**: Similar to feature selection methods, Data Minimizer evaluates the model’s performance as features are iteratively removed. By identifying features that have minimal impact on model accuracy, it highlights areas where data minimization can improve privacy without sacrificing effectiveness.

- **Privacy Risk Score**: This metric quantifies the likelihood that a specific input sample originates from the model's training set, based on the model's responses to that sample. Privacy Risk Score helps to identify which samples present the highest risk to be classified as training data, or in general, how risky is the model in terms of data privacy with respect to our training data.

In [ ]:
from holisticai.security.commons import DataMinimizer
from holisticai.security.metrics import data_minimization_score

dmin = DataMinimizer(proxy=proxy, selector_types=["Percentile", "Variance"])
dmin.fit(train['X'], train['y'])

y_pred_test = proxy.predict(test['X'])
y_pred_test_dm = dmin.predict(test['X'])


data_minimization_score(test['y'], y_pred_test, y_pred_test_dm)

In [ ]:
from holisticai.security.metrics import privacy_risk_score

target_shadow = train.train_test_split(test_size=0.4, random_state=42)

X_shadow_train = target_shadow['test']['X']
y_shadow_train = target_shadow['test']['y']

shadow_model = RandomForestClassifier(random_state=42)
shadow_model.fit(X_shadow_train, y_shadow_train)

shadow_train_probs = shadow_model.predict_proba(X_shadow_train)
shadow_test_probs = shadow_model.predict_proba(test['X'])
target_train_probs = proxy.predict_proba(train['X'])
target_test_probs = proxy.predict_proba(test['X'])

risk_score_train = privacy_risk_score((shadow_train_probs, y_shadow_train), (shadow_test_probs, test['y']), (target_train_probs, train['y']))
print("Mean Privacy Risk Score for train: ", risk_score_train.mean())

Given that this metric estimates an individual sample’s probability of being in the target model’s training set. By calculating the mean Privacy Risk Score across all samples, we can obtain a global measure of the model’s privacy risk. In an ideal scenario, the Privacy Risk Score should be close to 0, indicating a low risk of data memorization, while higher values suggest potential vulnerabilities to membership inference attacks.

# Measuring Robustness

Robustness metrics are measures used to evaluate how well machine learning models maintain performance under varying conditions, such as data shifts, adversarial inputs, or environmental changes. These metrics help assess a model’s resilience and reliability in real-world applications.


Accuracy Degradation Factor (ADF) is a robustness metric that detects the first point of significant accuracy drop as a dataset is gradually reduced, allowing for the prediction of performance shifts due to changing data distributions. Here is an exampel how to use the metric:

In [ ]:
from holisticai.robustness.metrics import (
    accuracy_degradation_profile,
    accuracy_degradation_factor,
)
import pandas as pd

results = accuracy_degradation_profile(test['X'], 
                                    test['y'], 
                                    y_test_pred, 
                                    n_neighbors = 50,
                                    step_size = 0.02,
                                    )
accuracy_degradation_factor(pd.DataFrame(results.data))

## Extra Example: Bias-Accuracy Tradeoff

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set_style("whitegrid")

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from holisticai.bias.metrics import equal_opportunity_diff, statistical_parity, disparate_impact
from sklearn.metrics import accuracy_score

In [ ]:
class BaseMetric:
    # BaseMetric class with typical inputs
    def __init__(self, model, train, test):
        self.model = model
        self.train = train
        self.test = test

class MyCustomMetric(BaseMetric):
    best_value = 0
    # You can define your own custom metric to be optimized                
    def compute(self):
        y_pred = self.model.predict(self.test['X'])
        acc = accuracy_score(self.test['y'], y_pred)
        di = disparate_impact(self.test['group_a'], self.test['group_b'], y_pred)
        
        error_accuracy = 1 - acc # min value: 0
        error_bias = abs(1-di) # min value: 0
        error  = error_accuracy + error_bias
        return error

class EqualOpportunity(BaseMetric):
    best_value = 0
    def compute(self):
        y_pred = self.model.predict(self.test['X'])
        eo = equal_opportunity_diff(self.test['group_a'], self.test['group_b'], y_pred, self.test['y'])
        return eo

class Accuracy(BaseMetric):
    best_value = 1
    def compute(self):
        y_pred = self.model.predict(self.test['X'])
        eo = accuracy_score(self.test['y'], y_pred)
        return eo

class DisparateImpact(BaseMetric):
    best_value = 1
    def compute(self):
        y_pred = self.model.predict(self.test['X'])
        di = disparate_impact(self.test['group_a'], self.test['group_b'], y_pred)
        return di

In [ ]:
from typing import Callable
from tqdm import tqdm

class MultiObjetiveOptimizer:
    models = {
        'LR_l2': LogisticRegression(penalty='l2', C=1.0, solver='liblinear', random_state=42),
        'LR_l1': LogisticRegression(penalty='l1', C=1.0, solver='liblinear', random_state=42),
        'RF_50': RandomForestClassifier(n_estimators=50, random_state=42),
        'RF_100': RandomForestClassifier(n_estimators=100, random_state=42),
        'RF_150': RandomForestClassifier(n_estimators=150, random_state=42),
        'DT_5': DecisionTreeClassifier(max_depth=5, random_state=42),
        'DT_10': DecisionTreeClassifier(max_depth=10, random_state=42),
        'DT_15': DecisionTreeClassifier(max_depth=15, random_state=42),
        'MLP_50': MLPClassifier(hidden_layer_sizes=(50,), random_state=42),
        'MLP_100': MLPClassifier(hidden_layer_sizes=(100,), random_state=42),
        'MLP_150': MLPClassifier(hidden_layer_sizes=(150,), random_state=42),
    }

    def optimize(self, train, test, metric_evaluation_class: Callable, metric_monitor_classes: list[Callable]):
        best_obj_score = np.inf
        history = []
        def objective_function(model, train, test):
            metric = metric_evaluation_class(model, train, test) 
            return np.abs(metric.best_value - metric.compute())     

        history = []
        for model_name, model in tqdm(self.models.items()):
            model = model
            model.fit(train['X'], train['y'])
            
            obj_score = objective_function(model, train, test)
             
            if best_obj_score > obj_score:
                best_obj_score = obj_score
                best_model_name = model_name
                best_model = model

            scores = {m.__name__:m(model, train, test).compute() for m in metric_monitor_classes}
            history.append({'model': model_name, 'obj_score':obj_score, **scores})

        return {
            'best_model_name': best_model_name,
            'best_model': best_model,
            'best_obj_score': best_obj_score,
            'history': history
        }

In [ ]:
optimizer = MultiObjetiveOptimizer()

metric_evaluation_class = MyCustomMetric
metric_monitor_classes = [EqualOpportunity, Accuracy, DisparateImpact]

result = optimizer.optimize(train, test, metric_evaluation_class, metric_monitor_classes)

## Best Model

In [ ]:
result['best_model_name'], result['best_obj_score']

## Some visualizations

In [ ]:
import pandas as pd

df = pd.DataFrame(result['history'])
df

In [ ]:
def plot_accuracy_vs_bias_with_manual_colorbar(df):
    fig, axes = plt.subplots(1, 2, figsize=(15, 6), sharey=False)
    
    # Scatter plot for Accuracy vs EqualOpportunity with color based on obj_score
    sc1 = axes[0].scatter(df['Accuracy'], df['EqualOpportunity'], 
                          c=df['obj_score'], cmap='coolwarm', s=100)
    axes[0].set_title('Accuracy vs Equal Opportunity')
    axes[0].set_xlabel('Accuracy')
    axes[0].set_ylabel('Equal Opportunity')
    
    # Annotate points with model names
    for i, model in enumerate(df['model']):
        axes[0].annotate(model, (df['Accuracy'][i], df['EqualOpportunity'][i]), fontsize=8, ha='right')

    # Scatter plot for Accuracy vs DisparateImpact with color based on obj_score
    sc2 = axes[1].scatter(df['Accuracy'], df['DisparateImpact'], 
                          c=df['obj_score'], cmap='coolwarm', s=100)
    axes[1].set_title('Accuracy vs Disparate Impact')
    axes[1].set_xlabel('Accuracy')
    axes[1].set_ylabel('Disparate Impact')

    # Annotate points with model names
    for i, model in enumerate(df['model']):
        axes[1].annotate(model, (df['Accuracy'][i], df['DisparateImpact'][i]), fontsize=8, ha='right')

    # Adding color bar with substantial padding on the right of the figure
    cbar = fig.colorbar(sc1, ax=axes, orientation='vertical', fraction=0.03)
    cbar.set_label('Objective Score (obj_score)')
    
    plt.show()

In [ ]:
plot_accuracy_vs_bias_with_manual_colorbar(df)